# Set up Spotify credentials

Before getting started you need:

* Spotify API permissions & credentials that could apply for [here](https://developer.spotify.com/). Simply log in, go to your “dashboard” and select “create client id” and follow the instructions. Spotify are not too strict on providing permissions so put anything you like when they ask for commercial application.

* Python module — spotipy — imported

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 1.2 MB/s eta 

In [ ]:
!pip install spotipy
import spotipy
#To access authorised Spotify data - https://developer.spotify.com/
from spotipy.oauth2 import SpotifyClientCredentials
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
import pandas as pd
import seaborn as sns
!pip install gradio
import gradio as gr
#https://gradio.app/docs/#i_slider
import matplotlib.pyplot as plt
import time
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 2.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
#Create Function for identifying your artist

def choose_artist(name_input, sp):
  results = sp.search(name_input)
  #result_1 = result['tracks']['items'][0]['artists']
  top_matches = []
  counter = 0
  #for each result item (max 10 I think)
  for i in results['tracks']['items']:
    #store current item
    current_item = results['tracks']['items'][counter]['artists']
    counter+=1
    #for each item in that search_term
    counter2 = 0
    for i in current_item:
      #append artist name to top_matches
      #I will need to append something to identify the correct match, please update once I know
      top_matches.append((current_item[counter2]['name'], current_item[counter2]['uri']))
      counter2+=1

  #remove duplicates by turning list into a set, then back into a list
  top_matches = list(set(top_matches))

  fuzzy_matches = []
  #normal list doesn't need len(range)
  for i in top_matches:
    #put ratio result in variable to avoid errors
    ratio = fuzz.ratio(name_input, i[0])
    #store as tuple but will need to increase to 3 to include uid
    fuzzy_matches.append((i[0], ratio, i[1]))
  #sort fuzzy matches by ratio score
  fuzzy_matches = sorted(fuzzy_matches, key=lambda tup: tup[1], reverse=True)
  #store highest tuple's attributes in chosen variables
  chosen = fuzzy_matches[0][0]
  chosen_id = fuzzy_matches[0][1]
  chosen_uri = fuzzy_matches[0][2]
  print("The results are based on the artist: ", chosen)
  return chosen, chosen_id, chosen_uri

In [ ]:
#Function to Pull all of your artist's albums
def find_albums(artist_uri, sp):
  sp_albums = sp.artist_albums(artist_uri, album_type='album', limit=50) #There's a 50 album limit
  album_names = []
  album_uris = []
  for i in range(len(sp_albums['items'])):
    #Keep names and uris in same order to keep track of duplicate albums
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
  return album_uris, album_names

In [ ]:
#Function to store all album details along with their song details
def albumSongs(album, sp, album_count, album_names, spotify_albums):
    spotify_albums[album] = {} #Creates dictionary for that specific album
    #Create keys-values of empty lists inside nested dictionary for album
    spotify_albums[album]['album_name'] = [] #create empty list
    spotify_albums[album]['track_number'] = []
    spotify_albums[album]['song_id'] = []
    spotify_albums[album]['song_name'] = []
    spotify_albums[album]['song_uri'] = []

    tracks = sp.album_tracks(album) #pull data on album tracks

    for n in range(len(tracks['items'])): #for each song track
        spotify_albums[album]['album_name'].append(album_names[album_count]) #append album name tracked via album_count
        spotify_albums[album]['track_number'].append(tracks['items'][n]['track_number'])
        spotify_albums[album]['song_id'].append(tracks['items'][n]['id'])
        spotify_albums[album]['song_name'].append(tracks['items'][n]['name'])
        spotify_albums[album]['song_uri'].append(tracks['items'][n]['uri'])

In [ ]:
#Add popularity category
def popularity(album, sp, spotify_albums):
    #Add new key-values to store audio features
    spotify_albums[album]['popularity'] = []
    #create a track counter
    track_count = 0
    for track in spotify_albums[album]['song_uri']:
        #pull audio features per track
        pop = sp.track(track)
        spotify_albums[album]['popularity'].append(pop['popularity'])
        track_count+=1

In [ ]:
def gradio_music_graph(client_id, client_secret, artist_name): #total_albums
  #Insert your credentials
  client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
  sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) #spotify object to access API
  #Choose your artist via input
  chosen_artist, ratio_score, artist_uri = choose_artist(artist_name, sp=sp)
  #Retrieve their album details
  album_uris, album_names = find_albums(artist_uri, sp=sp)
  #Create dictionary to store all the albums
  spotify_albums = {}
  #Album count tracker
  album_count = 0
  for i in album_uris: #for each album
      albumSongs(i, sp=sp, album_count=album_count, album_names=album_names, spotify_albums=spotify_albums)
      print("Songs from " + str(album_names[album_count]) + " have been added to spotify_albums dictionary")
      album_count+=1 #Updates album count once all tracks have been added

  #To avoid it timing out
  sleep_min = 2
  sleep_max = 5
  start_time = time.time()
  request_count = 0
  #Update albums with popularity scores
  for album in spotify_albums:
      popularity(album, sp=sp, spotify_albums=spotify_albums)
      request_count+=1
      if request_count % 5 == 0:
          # print(str(request_count) + " playlists completed")
          time.sleep(np.random.uniform(sleep_min, sleep_max))
          # print('Loop #: {}'.format(request_count))
          # print('Elapsed Time: {} seconds'.format(time.time() - start_time))

  #Create song dictonary to convert into Dataframe
  dic_df = {}

  dic_df['album_name'] = []
  dic_df['track_number'] = []
  dic_df['song_id'] = []
  dic_df['song_name'] = []
  dic_df['song_uri'] = []
  dic_df['popularity'] = []

  for album in spotify_albums:
      for feature in spotify_albums[album]:
          dic_df[feature].extend(spotify_albums[album][feature])
  #Convert into dataframe

  df = pd.DataFrame.from_dict(dic_df)
  df = df.sort_values(by='popularity')
  df = df.drop_duplicates(subset=['song_id'], keep=False)

#Parameters of a Original Plot, unfortunately, no longer works (for now)
  # sns.set_style('ticks')

  # fig, ax = plt.subplots()
  # fig.set_size_inches(11, 8)
  # ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
  # plt.tight_layout()

  # sns.boxplot(x=df["album_name"], y=df["popularity"], ax=ax)
  # fig.savefig('artist_popular_albums.png')
  # plt.show()

  return df

plot = gr.ScatterPlot(x="album_name", y="popularity", width=600, height=350, title="Popular Songs By Album Box Plot Distribution on Spotify")
#Interface will include these buttons based on parameters in the function with a dataframe output

#Ignore, old code
#music_plots = gr.Interface(gradio_music_graph, ["text", "text", "text"],
#                           ["dataframe", "plot"], title="Popular Songs By Album Box Plot Distribution on Spotify", description="Using your Spotify API Access from https://developer.spotify.com/ you can see your favourite artist's most popular albums on Spotify")

music_plots = gr.Interface(fn=gradio_music_graph, inputs=["text","text","text"], outputs=plot)


music_plots.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://052cd3b23b87e8fa87.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


The results are based on the artist:  Future
Songs from WE STILL DON'T TRUST YOU have been added to spotify_albums dictionary
Songs from WE DON'T TRUST YOU have been added to spotify_albums dictionary
Songs from I NEVER LIKED YOU have been added to spotify_albums dictionary
Songs from I NEVER LIKED YOU (Instrumental) have been added to spotify_albums dictionary
Songs from Pluto x Baby Pluto (Deluxe) have been added to spotify_albums dictionary
Songs from Pluto x Baby Pluto have been added to spotify_albums dictionary
Songs from High Off Life have been added to spotify_albums dictionary
Songs from SAVE ME have been added to spotify_albums dictionary
Songs from Future Hndrxx Presents: The WIZRD have been added to spotify_albums dictionary
Songs from Future & Juice WRLD Present... WRLD ON DRUGS have been added to spotify_albums dictionary
Songs from BEASTMODE 2 have been added to spotify_albums dictionary
Songs from SUPERFLY (Original Motion Picture Soundtrack) have been added to spotify_